# Generating Multi-label Discrete Electronic Health Records using Generative Adversarial Networks
This paper deals with the problem of privacy when using electronic health records (EHR) data in medical research. Synthesizing this kind of data can mitigate this risk so they try to generate EHR data with GANs.

The main problem is that this type of data consists of discrete variables, both binary and count features.

## Model
This paper presents an approach using a combination of an autoencoder and a GAN.

Generator G is trained on error signal from discriminator D so the chain of composed functions must be continuous in order for a gradient to flow so the output of G is also continuous.

Instead they let G generate data in the same space as the hidden layer of an autoencoder. The hidden layer of the autoencoder should capture the important features of the (discrete) data.

They can then use the decoder of the autoencoder to decode the output of G to get data in the discrete data space.

### Architecture

```
X --> Encoder ----> Decoder --> X'

Z --> Generator --> Decoder --> X_z --> D --> real/fake

                                X ----> D --> real/fake
```

### Training
First they pretrain the autoencoder using
* Mean squared error for count features
* Cross entropy loss for binary features

Then they train G and D of the GAN part of the model by
1. Update D for k steps
    1. $X \sim p_{data}$, $Z \sim p_z$
    2. $X_z = decoder(G(Z))$
    3. $\overline{X_z} = mean(X_z)$
    4. $\overline{X} = mean(X)$
    4. Update $\theta_D$ by maximizing $mean \left( log\ D(X, \overline{X}) + log\ (1 - D(X_z, \overline{X}_z) \right)$
2. Update G and decoder
    1. $Z \sim p_z$
    2. $X_z = decoder(G(Z))$
    3. $\overline{X_z} = mean(X_z)$
    4. Update $\theta_G, \theta_{decoder}$ by maximizing $mean \left( log\ D(X_z, \overline{X}_z) \right)$

### Details
* AE: Standard MLP
* G: ReLU activations in every layer except tanh in last
* G: Batch normalization (no biases needed)
* G: Shortcut connections (in addition to nonlinearity and BN, add input unchanged to output)
* D: Standard MLP with dropout
* D: k = 2 in their experiments

## Minibatch averaging
The other contribution of this paper is minibatch averaging which is another attempt at reducing risk of mode collapse (i.e. generator mapping many different Z to same output).

Same reasoning as minibatch discrimination from "Improved techniques for training GANs", i.e. let D see the whole minibatch when classifying one sample.

Minibatch averaging simply takes the mean of the samples in the minibatch and concatenates it to every sample to classify. Real and fake samples are handled separately.

Supposedly better than minibatch discrimination because 
* For binary features the mean corresponds to maximum likelihood estimate of bernoulli distribution parameters $p$ and passing this information to D makes it easier to decide since in the mode collapse case $p$ will be very close to either 0 or 1 for fake samples and thus easy for D to tell that it's fake.
* For count variables the mean corresponds to an estimate of the binomial distribution which should also better guide G to generate more diverse and real looking samples.

Minibatch averaging is also very easy to implement and fast.

They also suggest using a larger minibatch (they use 1000) than usual to better capture the statistics of real data.

## Thoughts
They don't seem to have both binary and count variables at the same time. They either define the model with binary or with count variables and define the losses thereafter. Can they be used at the same time? Like, can we have the first k features be binary and the rest are count variables for example and then create a combined loss for both?